In [ ]:
from dotenv import load_dotenv; load_dotenv()
from openai import OpenAI
from pathlib import Path
import datetime as dt, json

client = OpenAI()
out = Path("../data/outputs"); out.mkdir(parents=True, exist_ok=True)
TS = dt.datetime.now().strftime("%Y%m%d_%H%M%S")



In [ ]:

r = client.responses.create(
    model="gpt-4o-mini",  # use a working model on your account
    input="Give me 3 AI features for vacation rental hosts (one line each)."
)
print(r.output_text)
(out / f"resp_{TS}.txt").write_text(r.output_text)




In [ ]:
r = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role":"user","content":"Give me 3 AI features for vacation rental hosts (one line each)."}],
    temperature=0.7,
)
text = r.choices[0].message.content
print(text)
(out / f"chat_{TS}.txt").write_text(text)
print("usage:", getattr(r, "usage", None))

In [ ]:
messages = [
    {"role":"system","content":"You are a concise product assistant."},
    {"role":"user","content":"Propose 3 features to reduce guest support load."}
]
r1 = client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=messages)

print("A1:", r1.choices[0].message.content)

messages.append({"role":"assistant","content": r1.choices[0].message.content})
messages.append({"role":"user","content":"Great. Turn #2 into a one-week sprint plan with tasks."})

r2 = client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=messages)

print("A2:", r2.choices[0].message.content)


In [ ]:
from openai import OpenAI
client = OpenAI()

# Turn 1
r1 = client.responses.create(
    model="gpt-4o-mini",
    instructions="You are a concise product assistant.",
    input="Propose 3 features to reduce guest support load.",
    store=True,  # keep state so we can chain
)
print(r1.output_text)

# Turn 2 (continues the same conversation)
r2 = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=r1.id,  # <-- carries context from r1
    input="Great. Turn #2 into a one-week sprint plan with tasks."
)
print(r2.output_text)


In [ ]:
prompt = "Write a 1-sentence positioning statement for a vacation rental AI co-pilot."
for t in [0.0, 0.3, 0.7, 1.0]:
    r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system","content":"Be crisp, avoid hype, <12 words."},
            {"role":"user","content": prompt},
        ],
        temperature=t,
    )
    print(f"\nTemp {t}: {r.choices[0].message.content}")


In [ ]:
r = client.chat.completions.create(
    model="gpt-4o-mini",
    response_format={"type":"json_object"},
    messages=[{"role":"user","content":"Return JSON with keys title, problem, impact(1-10) for 1 feature idea."}],
    temperature=0.3
)
data = json.loads(r.choices[0].message.content)
data

In [ ]:
messages = [
    {"role":"system","content":"You are a concise product assistant."},
    {"role":"user","content":"Propose 3 features to reduce guest support load."}
]
r1 = client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=messages)

print("A1:", r1.choices[0].message.content)

messages.append({"role":"assistant","content": r1.choices[0].message.content})
messages.append({"role":"user","content":"Return JSON with keys title, problem, impact(1-10) for 1 feature idea."})

r2 = client.chat.completions.create(
    model="gpt-4o-mini",
    response_format={"type":"json_object"}, 
    messages=messages,
    temperature=0.3)


data = json.loads(r2.choices[0].message.content)
data

In [ ]:
def save_json(name, obj):
    p = out / f"{name}_{TS}.json"
    p.write_text(json.dumps(obj, indent=2))
    return p

log = {
    "ts": TS,
    "patterns": ["responses", "chat", "multi_turn", "temp_sweep", "json_lite"],
}
print("Saved log to:", save_json("day2_patterns_log", log))


In [ ]:
from dotenv import load_dotenv; load_dotenv()
from openai import OpenAI
from pathlib import Path
from datetime import datetime
import json, jsonschema, pandas as pd

client = OpenAI()
OUT = Path("../data/outputs"); OUT.mkdir(parents=True, exist_ok=True)
TS = datetime.now().strftime("%Y%m%d_%H%M%S")

def save_json(path: Path, obj: dict):
    path.write_text(json.dumps(obj, indent=2))
    return path.resolve()


In [ ]:
SCHEMA = {
    "type": "object",
    "properties": {
        "topic": {"type": "string"},
        "ideas": {
            "type": "array",
            "items": {
                "type":"object",
                "properties":{
                    "title":  {"type":"string"},
                    "problem":{"type":"string"},
                    "impact": {"type":"integer", "minimum":1, "maximum":10},
                    "effort": {"type":"integer", "minimum":1, "maximum":10}
                },
                "required": ["title","problem","impact","effort"]
            }
        }
    },
    "required": ["topic","ideas"]
}
SCHEMA


In [ ]:
from dotenv import load_dotenv; load_dotenv()
from openai import OpenAI
import json, jsonschema
from datetime import datetime
from pathlib import Path

client = OpenAI()
OUT = Path("../data/outputs"); OUT.mkdir(parents=True, exist_ok=True)
TS = datetime.now().strftime("%Y%m%d_%H%M%S")

topic = "Vacation rental software"
n = 5

# Wrap your schema for Structured Outputs
structured_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "ideas_schema",
        "schema": SCHEMA,     # <- the dict you defined earlier
        "strict": True        # enforce the schema
    }
}

instructions = "You are a concise product ideation assistant."
prompt = f'Generate exactly {n} ideas for topic "{topic}". Return JSON only—no prose.'

resp = client.responses.create(
    model="gpt-4o-mini",      # use a model available on your account
    instructions=instructions,
    input=prompt,
    response_format=structured_format,
    temperature=0.2,
)

raw = resp.output_text
print("RAW preview:", raw[:300], "...\n")

data = json.loads(raw)                # parse JSON
jsonschema.validate(data, SCHEMA)     # validate against your schema
print("✅ Valid JSON with", len(data["ideas"]), "ideas")

# Save
out_json = OUT / f"ideas_structured_{TS}.json"
out_json.write_text(json.dumps(data, indent=2))
out_json

